In [22]:
import pandas as pd
import numpy as np
import pymongo
import itertools
import csv
import json
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["flick"]

### Initialize

In [23]:
mycol = mydb["genres_datas"]
mycol.drop()
genres_df=pd.read_csv('genres.csv')
genre_json=genres_df.to_json(orient='records')
genre_json=eval(genre_json)
genre_data=genre_json
x = mycol.insert_many(genre_data)


mycol=mydb["movies_datas"]
mycol.drop()
csvfile = open('movies.csv', 'r',encoding='utf-8')
reader = csv.DictReader( csvfile )
header=["movieId","imdb_link","poster","title","imdb_score","genres"]
for each in reader:
    row={}
    for field in header:
        row[field]=each[field]

    mycol.insert_one(row)

### New review

In [26]:
#inserting new review
uid=2
movieId=113228
rating=4
rating=float(rating)
mycol = mydb["reviews"]
record={ "uid": uid, "movieId": movieId,"rating":rating }
mycol.insert_one(record)
#generating userInput table
userInput=[]
for x in mycol.find({"uid":uid},{ "_id": 0}):
    userInput.append(x)
userInput=pd.DataFrame(userInput)
userInput=userInput.drop(['uid'], axis = 1) 
#fetching genres data from database
mycol = mydb["genres_data"]
data=[]
for x in mycol.find({},{"_id":0}):
    data.append(x)
genres_df=pd.DataFrame(data)
#generating userProfile
userGenre=genres_df[genres_df['movieId'].isin(userInput['movieId'].tolist())]
userGenre.drop('movieId',1,inplace=True)
userGenre.reset_index(drop=True)
userProfile = userGenre.transpose().dot(userInput.rating.values)
#getting new recommendations
genreTable=genres_df.copy()
genreTable.set_index('movieId',inplace=True)
recommend_df=((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommend_df.sort_values(ascending=False,inplace=True)
mycol = mydb["user_recommendation_data"]
recommendation_json=eval(recommend_df.to_json())
recommendation_data={"uid":uid,"recommendation_data":recommendation_json}
mycol.delete_one({"uid":uid})
x = mycol.insert_one(recommendation_data)
recommend_df.head()

movieId
18515     0.961538
79477     0.961538
177858    0.961538
34902     0.961538
111543    0.961538
dtype: float64

###    Recommendation

In [28]:
mycol = mydb["user_recommendation_data"]
x=mycol.find_one({"uid":2},{"_id":0})
data=x['recommendation_data']
# recommend_df=pd.Series(data)
# recommend_df.head()
data=dict(itertools.islice(data.items(),5))
movies_id=list(data.keys())

mycol=mydb['movies_data']
movies_data=[]
for i in mycol.find({},{"_id":0}):
    if(str(i["movieId"]) in movies_id):
        movies_data.append(i)
movies_data

[{'movieId': '111543',
  'imdb_link': 'http://www.imdb.com/title/tt111543',
  'poster': 'https://images-na.ssl-images-amazon.com/images/M/MV5BMjMzMjAxNTYyMF5BMl5BanBnXkFtZTYwMzE4ODc4._V1_UY268_CR2,0,182,268_AL_.jpg',
  'title': 'Little Indian, Big City (1994)',
  'imdb_score': '5.6',
  'genres': 'Adventure|Comedy|Romance'},
 {'movieId': '79477',
  'imdb_link': 'http://www.imdb.com/title/tt79477',
  'poster': 'https://images-na.ssl-images-amazon.com/images/M/MV5BMjE2MzYxMjMyN15BMl5BanBnXkFtZTcwMjI0NjYxMQ@@._V1_UX182_CR0,0,182,268_AL_.jpg',
  'title': 'A Little Romance (1979)',
  'imdb_score': '7.5',
  'genres': 'Adventure|Comedy|Romance'},
 {'movieId': '177858',
  'imdb_link': 'http://www.imdb.com/title/tt177858',
  'poster': 'https://images-na.ssl-images-amazon.com/images/M/MV5BMTIwMTM0MzMzNF5BMl5BanBnXkFtZTcwMTczNTUyMQ@@._V1_UY268_CR4,0,182,268_AL_.jpg',
  'title': 'In July (2000)',
  'imdb_score': '7.8',
  'genres': 'Adventure|Comedy|Romance'}]